In [1]:
%pip install pandas
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, f_classif, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, LogisticRegression
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
df = pd.read_csv("./datasets/train/train.csv") # для train

In [6]:

# Удаляем индекс, так как он не несёт информации для модели
X = df.drop(['index', 'target'], axis=1)  # Все фичи (feature_1 ... feature_1366)
y = df['target']  # Целевая переменная

X, y

(        feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
 0        0.128367        1.0        1.0        1.0        1.0        4.0   
 1        0.258685        1.0        1.0        1.0        0.0        7.0   
 2        0.407738        1.0        1.0        1.0        2.0        7.0   
 3        0.439134        1.0        1.0        1.0        1.0        2.0   
 4        0.590481        1.0        1.0        1.0        0.0        1.0   
 ...           ...        ...        ...        ...        ...        ...   
 177853   0.660324        1.0        1.0        1.0        0.0        1.0   
 177854   0.222214        0.0        1.0        1.0        1.0        4.0   
 177855   0.545399        0.0        1.0        1.0        0.0        2.0   
 177856   0.393150        1.0        1.0        1.0        0.0        1.0   
 177857   0.395272        1.0        1.0        1.0        1.0        2.0   
 
         feature_6  feature_7  feature_8  feature_9  ...  feature_1357  \


In [7]:
indexes = df['index']

In [6]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=200)),
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

X_selected = pipe.fit_transform(X, y)
selector = pipe.named_steps['kbest']

print(f"Исходное число признаков: {X.shape[1]}")
print(f"Число признаков после отбора: {X_selected.shape[1]}")

Исходное число признаков: 1367
,Число признаков после отбора: 200


In [7]:
# 2. Масштабирование данных (важно для линейной регрессии)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.3, random_state=42
)


In [ ]:
# 4. Создание и обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# 5. Предсказание и оценка качества
y_pred = model.predict(X_test)
# y_pred_proba_lin_reg = model.decision_function(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
# нормализуем значения для roc_auc
y_proba = 1 / (1 + np.exp(-y_pred)) # преобразование через сигмойду
roc_auc = roc_auc_score(y_test, y_proba)

print(f"\nMSE: {mse:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

# (Опционально) Вывод коэффициентов модели
if X_selected.shape[1] <= 20:  # Не выводить для слишком большого числа фич
    print("\nКоэффициенты модели:")
    # for feature, coef in zip(X.columns[selector.get_support()], model.coef_):
    #     print(f"{feature}: {coef:.4f}")
else:
    print(f"\nМодель имеет {X_selected.shape[1]} коэффициентов (слишком много для вывода).")


,MSE: 0.0171
,R2 Score: 0.0016
,ROC-AUC Score: 0.6536
,
,Модель имеет 400 коэффициентов (слишком много для вывода).


In [42]:
y_pred

array([ 0.01629677,  0.00887584,  0.00911322, ...,  0.0150569 ,
       -0.0037384 , -0.00022041], shape=(53358,))

Так.... Из полученных метрик следует что вроде бы по MSE результат довольно терпимый и среднеквадратичная ошибка не слишком большая.
Касаемо метрики ROC-AUC Score получаем 0.6536 что является лучше чем убычное угадывание(подбрасывание монетки).
Из инересного мы не должны использовать LinearRegression, так как она может выдавать значения за пределами [0, 1], но после регуляризации значений, они могут быть использованиы для вычисления метрики. 

Попробуем RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)


In [13]:
y_pred_proba

array([0.  , 0.04, 0.03, ..., 0.  , 0.03, 0.  ], shape=(53358,))

In [47]:
print(f"ROC_AUC_SCORE is {roc_auc:.4f}")

ROC_AUC_SCORE is 0.6261


Прогресс, модель достигла случайного угадывания
(последние данные ROC_AUC_SCORE is 0.6261, это даже лучше чем просто уадывание по среднему значению)

In [32]:
mse = mean_squared_error(y_test, y_pred_proba)
print(f'MSE: {mse:.2f}')

MSE: 0.02


Небольшой апдейт для структуры модели

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier())
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)


In [33]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.6221


А что если заменить RandomForsetClassifier на LogisticRegression

In [ ]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)

In [37]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.6473


Вау, метрика подросла)


Теперь попробуем GradientBoostingClassifier


In [13]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', GradientBoostingClassifier())
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)

In [14]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.6791


Попробуем еще одну модель SVC

In [6]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', SVC(probability=True, kernel='rbf', C=1.0, decision_function_shape='ovr'))
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)

In [7]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.5873


Начинаем работу с XGBoost

Подготовим данные для него

In [8]:
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('scaler', StandardScaler()),
    # ('kbest', SelectKBest(f_classif, k=500)),
])

X_selected = pipe.fit_transform(X, y)

In [9]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, interaction_only=True)
X_poly = poly.fit_transform(X_selected)

MemoryError: Unable to allocate 1.06 TiB for an array with shape (177858, 821122) and data type float64

Для полиномиальных признаков

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X_poly, y, test_size=0.2, random_state=42
)

NameError: name 'X_poly' is not defined

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42
)

Попытаемся создать полиномиальные признаки

Присутствуют импорты библиотек и моделей, чтобы лучше понимать что происходит и не возвращаться каждый раз наверх

Пипец долго считает)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

model = XGBClassifier(objective='binary:logistic', eval_metric='auc', tree_method='hist', device='cuda')

param_grid = {
    'n_estimators': [500, 1000, 1500, 1700],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
    'reg_alpha': [0, 1],
    'reg_lambda': [0, 1]
}

grid = GridSearchCV(model, param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

print(f"Лучшие параметры: {grid.best_params_}")
print(f"Лучший ROC-AUC: {grid.best_score_:.4f}")

In [9]:
import optuna
from sklearn.metrics import roc_auc_score

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }
    
    model = XGBClassifier(**params, objective='binary:logistic', 
                          scale_pos_weight=10, device='cuda', tree_method='hist', eval_metric='auc')
    model.fit(X_train, y_train)
    y_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_proba)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)
print(f"Лучший ROC-AUC: {study.best_value:.4f}")
print(f"Лучшие параметры: {study.best_params}")

[I 2025-04-24 15:47:06,315] A new study created in memory with name: no-name-ad40d7ca-dec2-4e7d-8ff9-560291bc2fde
,  0%|          | 0/50 [00:00<?, ?it/s]c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:729: UserWarning: [15:48:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
,Potential solutions:
,- Use a data structure that matches the device ordinal in the booster.
,- Set the device for booster before call to inplace_predict.
,
,This warning will only be shown once.
,
,  return func(**kwargs)
,Best trial: 0. Best value: 0.719963:   2%|▏         | 1/50 [01:39<1:20:56, 99.11s/it]

[I 2025-04-24 15:48:45,425] Trial 0 finished with value: 0.7199628349060889 and parameters: {'n_estimators': 1125, 'max_depth': 8, 'learning_rate': 0.02241833424878355, 'subsample': 0.8096449737437666, 'colsample_bytree': 0.7073004384635065, 'reg_alpha': 6.533106038440083, 'reg_lambda': 9.262728769416047}. Best is trial 0 with value: 0.7199628349060889.


Best trial: 0. Best value: 0.719963:   4%|▍         | 2/50 [03:13<1:16:53, 96.12s/it]

[I 2025-04-24 15:50:19,449] Trial 1 finished with value: 0.6954017664553149 and parameters: {'n_estimators': 2776, 'max_depth': 4, 'learning_rate': 0.18976614883120557, 'subsample': 0.9024856139308948, 'colsample_bytree': 0.7919114539903954, 'reg_alpha': 0.8403179895039259, 'reg_lambda': 5.852740218876109}. Best is trial 0 with value: 0.7199628349060889.


Best trial: 0. Best value: 0.719963:   6%|▌         | 3/50 [05:59<1:40:17, 128.03s/it]

...

[I 2025-04-24 16:27:41,033] Trial 47 finished with value: 0.7189606258496759 and parameters: {'n_estimators': 849, 'max_depth': 3, 'learning_rate': 0.08288731562124517, 'subsample': 0.750959391959956, 'colsample_bytree': 0.7632600075153755, 'reg_alpha': 3.608543800387094, 'reg_lambda': 8.48348611406188}. Best is trial 11 with value: 0.7267751825365547.


Best trial: 11. Best value: 0.726775:  98%|█████████▊| 49/50 [41:00<00:21, 21.05s/it]

[I 2025-04-24 16:28:06,352] Trial 48 finished with value: 0.7146341268455808 and parameters: {'n_estimators': 1100, 'max_depth': 4, 'learning_rate': 0.061597749611161115, 'subsample': 0.7145414151320666, 'colsample_bytree': 0.9852409588197536, 'reg_alpha': 2.082755347179272, 'reg_lambda': 8.9863625202634}. Best is trial 11 with value: 0.7267751825365547.


Best trial: 11. Best value: 0.726775: 100%|██████████| 50/50 [41:31<00:00, 49.83s/it]

[I 2025-04-24 16:28:37,791] Trial 49 finished with value: 0.7100451569784157 and parameters: {'n_estimators': 828, 'max_depth': 6, 'learning_rate': 0.09272990190140357, 'subsample': 0.7363564816005026, 'colsample_bytree': 0.9528118969636823, 'reg_alpha': 3.1682063284417445, 'reg_lambda': 8.261279961517689}. Best is trial 11 with value: 0.7267751825365547.
,Лучший ROC-AUC: 0.7268
,Лучшие параметры: {'n_estimators': 653, 'max_depth': 6, 'learning_rate': 0.02176645480492359, 'subsample': 0.7021475108158397, 'colsample_bytree': 0.9160694536741326, 'reg_alpha': 2.5180991485275133, 'reg_lambda': 6.366469735868886}


In [10]:
best_params = study.best_params
best_params

NameError: name 'study' is not defined

Был сбой, но best_params удалось сохранить в save.txt так что скопирую их оттуда

In [13]:
best_params = {
    'n_estimators': 653,
    'max_depth': 6,
    'learning_rate': 0.02176645480492359,
    'subsample': 0.7021475108158397,
    'colsample_bytree': 0.9160694536741326,
    'reg_alpha': 2.5180991485275133,
    'reg_lambda': 6.366469735868886
}

In [14]:
# generate output data
X_true_test_df = pd.read_csv('./datasets/test/test.csv')
X_true_test = X_true_test_df.drop(columns=['index'])
indexes = X_true_test_df['index']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [16]:
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('scaler', StandardScaler()),
    ('model', XGBClassifier(**best_params, objective='binary:logistic', 
                          scale_pos_weight=10, device='cuda', tree_method='hist', eval_metric='auc')),
    
    # ('kbest', SelectKBest(f_classif, k=500)),
])

pipe.fit(X_train, y_train)

y_train_pred_proba = pipe.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_train_pred_proba)
print(f'ROC-AUC is {roc_auc:.4f}')

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:729: UserWarning: [17:46:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


ROC-AUC is 0.7253


In [17]:
y_test_pred_proba = pipe.predict_proba(X_true_test)[:, 1]
y_test_pred_proba

array([0.06112183, 0.09746365, 0.03243464, ..., 0.03522414, 0.13069998,
       0.02850325], shape=(76225,), dtype=float32)

In [20]:
output = pd.concat([indexes, pd.Series(y_test_pred_proba)], axis=1)
print(output)
output.to_csv("output.csv", index=False)

        index         0
0      124365  0.061122
1      177140  0.097464
2      106632  0.032435
3       17630  0.018898
4       97189  0.013972
...       ...       ...
76220   55074  0.046130
76221   12771  0.126945
76222   80999  0.035224
76223   31517  0.130700
76224  250238  0.028503

[76225 rows x 2 columns]


In [12]:
model = XGBClassifier(**best_params, objective='binary:logistic', 
                          scale_pos_weight=10, device='cuda', tree_method='hist', eval_metric='auc')


In [ ]:
X_true_test = pd.read_csv('./datasets/test/test.csv')
X_true_test_selected = pipe.fit

,index,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_1357,feature_1358,feature_1359,feature_1360,feature_1361,feature_1362,feature_1363,feature_1364,feature_1365,feature_1366
0,124365,0.719921,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,177140,0.123122,0.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,106632,0.384612,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
3,17630,0.452527,0.0,1.0,1.0,1.0,4.0,5.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,97189,0.379611,1.0,1.0,1.0,2.0,11.0,3.0,4.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76220,55074,0.325234,0.0,1.0,1.0,1.0,3.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
76221,12771,0.227776,1.0,1.0,1.0,1.0,3.0,3.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
76222,80999,0.459919,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
76223,31517,0.667716,1.0,1.0,1.0,0.0,2.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [15]:
model.fit(X_train, y_train)
y_train_pred = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_train_pred)
print(f'ROC-AUC is {roc_auc:.4f}')

ROC-AUC is 0.7268


In [11]:
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    # ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier(  # XGBoost с настройками для ROC-AUC
        objective='binary:logistic',
        eval_metric='auc',
        n_estimators=2500,
        max_depth=6,
        learning_rate=0.01,
        subsample=0.8,
        colsample_bytree=0.4,
        reg_alpha=1,
        reg_lambda=1,
        random_state=42,
        tree_method='hist',
        device="cuda"
    ))
])

pipe.fit(X_train, y_train)
y_proba = pipe.predict_proba(X_test)[:, 1]
roc = roc_auc_score(y_test, y_proba)
roc

KeyboardInterrupt: 